In [3]:
import numpy as np 
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [14]:
import numpy as np
import pandas as pd

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import csv

def scrape_ambitionbox_top_companies(pages=25, output_file="top_companies.csv"):
    options = Options()
    # Comment this line if you want to see the browser window
    # options.add_argument("--headless")

    # Anti-bot headers
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("start-maximized")
    options.add_argument("disable-infobars")
    options.add_argument("--disable-extensions")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.5735.110 Safari/537.36")

    driver = webdriver.Chrome(options=options)
    all_data = []

    for page in range(1, pages + 1):
        url = f"https://www.ambitionbox.com/list-of-companies?page={page}"
        print(f"Scraping page {page}...")
        driver.get(url)
        time.sleep(6)

        soup = BeautifulSoup(driver.page_source, "html.parser")
        cards = soup.find_all("div", class_="companyCardWrapper")

        if not cards:
            print("⚠️ No companies found on page. Skipping.")
            continue

        for card in cards:
            name = card.find("meta", {"itemprop": "name"})
            name = name.get("content", "N/A") if name else "N/A"

            rating_tag = card.find("div", class_="rating_text")
            rating = rating_tag.text.strip() if rating_tag else "N/A"

            review_tag = card.find("span", class_="companyCardWrapper__companyRatingCount")
            reviews = review_tag.text.strip() if review_tag else "N/A"

            location_tag = card.find("span", class_="companyCardWrapper__interLinking")
            location = location_tag.text.strip() if location_tag else "N/A"

            all_data.append({
                "Company": name,
                "Rating": rating,
                "Reviews": reviews,
                "Industry & Location": location
            })

    driver.quit()

    # Save to CSV
    keys = ["Company", "Rating", "Reviews", "Industry & Location"]
    with open(output_file, "w", newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=keys)
        writer.writeheader()
        writer.writerows(all_data)

    print(f"\nScraping complete! Saved {len(all_data)} companies to {output_file}")

# Run it
scrape_ambitionbox_top_companies()


Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
Scraping page 16...
Scraping page 17...
Scraping page 18...
Scraping page 19...
Scraping page 20...
Scraping page 21...
Scraping page 22...
Scraping page 23...
Scraping page 24...
Scraping page 25...

✅ Scraping complete! Saved 500 companies to top_companies.csv


In [15]:
df=pd.read_csv(r"C:\Users\adity\Desktop\gemma\top_companies.csv")

In [16]:
df.head()

,Company,Rating,Reviews,Industry & Location
0,TCS,3.6,(98.1k),IT Services & Consulting | Bangalore / Bengalu...
1,Accenture,3.8,(63.3k),IT Services & Consulting | Bangalore / Bengalu...
2,Wipro,3.7,(57.3k),IT Services & Consulting | Bangalore / Bengalu...
3,Cognizant,3.7,(54.4k),IT Services & Consulting | Hyderabad / Secunde...
4,Capgemini,3.7,(46.1k),IT Services & Consulting | Bangalore / Bengalu...


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import re

def slugify(name):
    slug = name.lower()
    slug = re.sub(r'[^a-z0-9\s-]', '', slug)
    slug = re.sub(r'\s+', '-', slug.strip())
    return slug

def get_interview_questions(company_name, max_questions=20):
    slug = slugify(company_name)
    url = f"https://www.ambitionbox.com/interviews/{slug}-interview-questions"

    options = Options()
    # Use visible browser to verify it works
    # comment next line to see browser
    # options.add_argument("--headless=new")
    options.add_argument("start-maximized")
    options.add_argument("user-agent=Mozilla/5.0")
    driver = webdriver.Chrome(options=options)

    try:
        driver.get(url)
        time.sleep(6)

        # Scroll to load more
        for _ in range(5):
            driver.execute_script("window.scrollBy(0, 1000);")
            time.sleep(1.5)

        soup = BeautifulSoup(driver.page_source, "html.parser")
    except Exception as e:
        print(f"Error loading {url}: {e}")
        driver.quit()
        return []

    driver.quit()

    results = []
    question_tags = soup.find_all("div", class_="topQuesAnsCard__details--question")

    for tag in question_tags:
        if len(results) >= max_questions:
            break
        span = tag.find("span", class_="topQuesAnsCard__details--answerHtmlWrapper")
        if span:
            q = span.get_text(strip=True)
            q = q.replace("Q.", "").strip()
            results.append((company_name, q))

    return results

# Test
questions = get_interview_questions("TCS")

for i, (company, q) in enumerate(questions, 1):
    print(f"{i}. {q}")


1. How proficient are you in Python?
2. How would you restructure your current CTC?
3. What is IPL in Mainframe?
4. What is Salesforce?
5. What are the steps to create and handle a custom exception in programming?
6. What annotations are used in a REST API?
7. What is the difference between the '==' operator and the '.equals()' method in Java?
8. What does error code 57 signify?
9. What does AVRD mean?
10. What are some effective strategies for stress management?


In [ ]:
import re
import csv
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

def slugify(name):
    slug = name.lower()
    slug = re.sub(r'[^a-z0-9\s-]', '', slug)
    slug = re.sub(r'\s+', '-', slug.strip())
    return slug

def get_interview_questions(company_name, max_questions=20):
    slug = slugify(company_name)
    url = f"https://www.ambitionbox.com/interviews/{slug}-interview-questions"

    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("start-maximized")
    options.add_argument("user-agent=Mozilla/5.0")
    driver = webdriver.Chrome(options=options)

    try:
        driver.get(url)
        time.sleep(6)
        for _ in range(5):
            driver.execute_script("window.scrollBy(0, 1000);")
            time.sleep(1.5)
        soup = BeautifulSoup(driver.page_source, "html.parser")
    except Exception as e:
        print(f"Error loading {url}: {e}")
        driver.quit()
        return []

    driver.quit()

    results = []
    question_tags = soup.find_all("div", class_="topQuesAnsCard__details--question")

    for tag in question_tags:
        if len(results) >= max_questions:
            break
        span = tag.find("span", class_="topQuesAnsCard__details--answerHtmlWrapper")
        if span:
            q = span.get_text(strip=True).replace("Q.", "").strip()
            results.append((company_name, q))

    return results

# Example company list (replace with all 500)
top_companies = ["TCS", "Accenture", "Wipro", "Infosys", "Cognizant"]

# Output file
with open("interview_questions_500_companies.csv", "w", newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["Company", "Question"])

    for idx, company in enumerate(top_companies, 1):
        print(f"[{idx}/{len(top_companies)}] Scraping: {company}")
        questions = get_interview_questions(company)
        if not questions:
            print(f"No questions found for {company}")
        for row in questions:
            writer.writerow(row)
        time.sleep(2)

print("Done. Data saved to interview_questions_500_companies.csv")


[1/5] Scraping: TCS
[2/5] Scraping: Accenture
[3/5] Scraping: Wipro
[4/5] Scraping: Infosys
[5/5] Scraping: Cognizant
✅ Done. Data saved to interview_questions_500_companies.csv
